# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

# Import Data & Libraries

In [36]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [37]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col):
    """Mengembalikan himpunan nilai unik pada kolom tertentu dari dataset"""
    return set([row[col] for row in rows])

# contoh penggunaan
training_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'jeruk'],
    ['oren', 'kasar', 130, 'jeruk'],
]

print("Nilai unik kolom 0 (warna):", unique_vals(training_data, 0))
print("Nilai unik kolom 1 (tekstur):", unique_vals(training_data, 1))
print("Nilai unik kolom 3 (label):", unique_vals(training_data, 3))

Nilai unik kolom 0 (warna): {'merah', 'oren', 'hijau'}
Nilai unik kolom 1 (tekstur): {'halus', 'kasar'}
Nilai unik kolom 3 (label): {'jeruk', 'apel'}


In [38]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    """Menghitung jumlah kemunculan setiap label di dataset (kolom terakhir)."""
    counts = {}
    for row in rows:
        # label diasumsikan ada di kolom terakhir
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts
# contoh penggunaan
training_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'jeruk'],
    ['oren', 'kasar', 130, 'jeruk'],
    ['hijau', 'halus', 160, 'apel'],
]

print("Jumlah tiap kelas:", class_counts(training_data))

Jumlah tiap kelas: {'apel': 3, 'jeruk': 2}


In [39]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    """Mengembalikan True jika value bertipe int atau float, False jika tidak."""
    return isinstance(value, int) or isinstance(value, float)

# contoh penggunaan
print(is_numeric(7))        # True (integer)
print(is_numeric(3.14))     # True (float)
print(is_numeric("apel"))   # False (string)
print(is_numeric([1, 2]))   # False (list)

True
True
False
False


In [40]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = ">=" if is_numeric(self.value) else "=="
        return f"Apakah kolom[{self.column}] {condition} {self.value}?"


# contoh penggunaan 1
training_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'jeruk'],
]

# Buat pertanyaan: apakah berat (kolom 2) >= 150?
q1 = Question(2, 150)
print(q1)  # tampilkan pertanyaan
print("Contoh 1 cocok?", q1.match(training_data[0]))  # True
print("Contoh 3 cocok?", q1.match(training_data[2]))  # False

# Buat pertanyaan: apakah warna (kolom 0) == 'hijau'?
q2 = Question(0, "hijau")
print(q2)
print("Contoh 2 cocok?", q2.match(training_data[1]))  # True
print("Contoh 3 cocok?", q2.match(training_data[2]))  # False

Apakah kolom[2] >= 150?
Contoh 1 cocok? True
Contoh 3 cocok? False
Apakah kolom[0] == hijau?
Contoh 2 cocok? True
Contoh 3 cocok? False


In [41]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    """Membagi dataset menjadi dua berdasarkan apakah baris cocok dengan pertanyaan atau tidak."""
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


# contoh penggunaan
training_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'jeruk'],
    ['hijau', 'kasar', 120, 'anggur'],
]

# Pertanyaan: apakah berat (kolom 2) >= 150?
q = Question(2, 150)

true_rows, false_rows = partition(training_data, q)

print("Pertanyaan:", q)
print("\nTrue rows (cocok):")
for row in true_rows:
    print(row)
print("\nFalse rows (tidak cocok):")
for row in false_rows:
    print(row)

Pertanyaan: Apakah kolom[2] >= 150?

True rows (cocok):
['merah', 'halus', 150, 'apel']
['hijau', 'halus', 170, 'apel']

False rows (tidak cocok):
['oren', 'kasar', 140, 'jeruk']
['hijau', 'kasar', 120, 'anggur']


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [42]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    """Menghitung Gini Impurity dari sebuah dataset"""
    counts = class_counts(rows)
    impurity = 1
    total = float(len(rows))
    for lbl in counts:
        prob_of_lbl = counts[lbl] / total
        impurity -= prob_of_lbl ** 2
    return impurity


# contoh penggunaan
training_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'jeruk'],
    ['hijau', 'kasar', 120, 'anggur'],
]

print("Jumlah kelas:", class_counts(training_data))
print("Gini impurity dataset:", gini(training_data))

# Kalau semua label sama, Gini = 0
pure_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'apel'],
]
print("Jumlah kelas (pure):", class_counts(pure_data))
print("Gini impurity pure dataset:", gini(pure_data))

Jumlah kelas: {'apel': 2, 'jeruk': 1, 'anggur': 1}
Gini impurity dataset: 0.625
Jumlah kelas (pure): {'apel': 3}
Gini impurity pure dataset: 0.0


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [43]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    """Menghitung information gain dari hasil split dataset."""
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)


# contoh penggunaan
training_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'jeruk'],
    ['hijau', 'kasar', 120, 'anggur'],
]

# hitung Gini awal (sebelum split)
current_uncertainty = gini(training_data)

# Pertanyaan: apakah berat (kolom 2) >= 150?
q = Question(2, 150)
true_rows, false_rows = partition(training_data, q)

gain = info_gain(true_rows, false_rows, current_uncertainty)

print("Pertanyaan:", q)
print("Gini awal:", current_uncertainty)
print("Jumlah data True:", len(true_rows), "-> Gini:", gini(true_rows))
print("Jumlah data False:", len(false_rows), "-> Gini:", gini(false_rows))
print("Information Gain:", gain)


Pertanyaan: Apakah kolom[2] >= 150?
Gini awal: 0.625
Jumlah data True: 2 -> Gini: 0.0
Jumlah data False: 2 -> Gini: 0.5
Information Gain: 0.375


In [44]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    """Mengembalikan information gain terbaik dan pertanyaan yang menghasilkan split tersebut."""
    best_gain = 0  # default
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # jumlah kolom fitur (tanpa label)

    for col in range(n_features):
        values = set([row[col] for row in rows])  # nilai unik di kolom ini

        for val in values:
            question = Question(col, val)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # hitung information gain
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # update jika gain lebih baik
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


# contoh penggunaan
training_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'jeruk'],
    ['hijau', 'kasar', 120, 'anggur'],
]

best_gain, best_question = find_best_split(training_data)
print("Pertanyaan terbaik:", best_question)
print("Information Gain terbaik:", best_gain)


Pertanyaan terbaik: Apakah kolom[2] >= 150?
Information Gain terbaik: 0.375


# Fungsi Decision Tree

In [45]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

# contoh penggunaan
training_data = [
    ['merah', 'halus', 150, 'apel'],
    ['hijau', 'halus', 170, 'apel'],
    ['oren', 'kasar', 140, 'jeruk'],
    ['hijau', 'kasar', 120, 'anggur'],
]

leaf_node = Leaf(training_data)
print("Prediksi di Leaf:", leaf_node.predictions)


Prediksi di Leaf: {'apel': 2, 'jeruk': 1, 'anggur': 1}


In [46]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self, question, true_branch, false_branch):
        self.question = question          # pertanyaan yang digunakan untuk split
        self.true_branch = true_branch    # subtree jika jawaban True
        self.false_branch = false_branch  # subtree jika jawaban False

# contoh penggunaan
q = Question(0, "hijau")
true_rows, false_rows = partition(training_data, q)

true_leaf = Leaf(true_rows)
false_leaf = Leaf(false_rows)
decision_node = Decision_Node(q, true_leaf, false_leaf)

print("Pertanyaan di node:", decision_node.question)
print("Cabang True:", decision_node.true_branch.predictions)
print("Cabang False:", decision_node.false_branch.predictions)


Pertanyaan di node: Apakah kolom[0] == hijau?
Cabang True: {'apel': 1, 'anggur': 1}
Cabang False: {'apel': 1, 'jeruk': 1}


In [47]:
# membangun decision tree secara rekursif
def build_tree(rows):
    # cari split terbaik
    gain, question = find_best_split(rows)

    # jika tidak ada gain, maka berhenti -> buat Leaf
    if gain == 0:
        return Leaf(rows)

    # bagi dataset berdasarkan pertanyaan terbaik
    true_rows, false_rows = partition(rows, question)

    # rekursif ke cabang true
    true_branch = build_tree(true_rows)

    # rekursif ke cabang false
    false_branch = build_tree(false_rows)

    # kembalikan decision node
    return Decision_Node(question, true_branch, false_branch)

# contoh penggunaan
tree = build_tree(training_data)
print("Tree berhasil dibangun!")


Tree berhasil dibangun!


In [48]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print(spacing + "Prediksi:", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# contoh penggunaan
print_tree(tree)


Apakah kolom[2] >= 150?
--> True:
  Prediksi: {'apel': 2}
--> False:
  Apakah kolom[2] >= 140?
  --> True:
    Prediksi: {'jeruk': 1}
  --> False:
    Prediksi: {'anggur': 1}


In [49]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
print("Hasil klasifikasi untuk ['hijau','halus',170,'?'] :", classify(['hijau','halus',170,None], tree))
print("Hasil klasifikasi untuk ['oren','kasar',140,'?'] :", classify(['oren','kasar',140,None], tree))


Hasil klasifikasi untuk ['hijau','halus',170,'?'] : {'apel': 2}
Hasil klasifikasi untuk ['oren','kasar',140,'?'] : {'jeruk': 1}


In [50]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for lbl in counts:
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

# contoh penggunaan
print("Prediksi data baru ['hijau','halus',170,'?'] :", print_leaf(classify(['hijau','halus',170,None], tree)))
print("Prediksi data baru ['oren','kasar',140,'?'] :", print_leaf(classify(['oren','kasar',140,None], tree)))


Prediksi data baru ['hijau','halus',170,'?'] : {'apel': '100%'}
Prediksi data baru ['oren','kasar',140,'?'] : {'jeruk': '100%'}


# Predict Using Decision Tree

In [51]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    print("Data:", row)
    print("Label asli:", row[-1])
    print("Prediksi:", print_leaf(classify(row, tree)))
    print()


Data: ['Green', 3, 'Apple']
Label asli: Apple
Prediksi: {'anggur': '100%'}

Data: ['Yellow', 4, 'Apple']
Label asli: Apple
Prediksi: {'anggur': '100%'}

Data: ['Red', 2, 'Grape']
Label asli: Grape
Prediksi: {'anggur': '100%'}

Data: ['Red', 1, 'Grape']
Label asli: Grape
Prediksi: {'anggur': '100%'}

Data: ['Yellow', 3, 'Lemon']
Label asli: Lemon
Prediksi: {'anggur': '100%'}

